<a href="https://colab.research.google.com/github/nindidooo/MODA/blob/main/AudioFingerprint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Cell

In [ ]:
#@title Setup the google drive
#Choose songs to split/analyse
songs = ['Vanilla-Ice-Ice-Ice-Baby', 'Queen-Under-Pressure']

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

root = '/content/gdrive/MyDrive/audio_files/Unmix Demo/'

Mounted at /content/gdrive


In [ ]:
#@title installs and configuration\
#Unmix
!pip install openunmix

#Magenta
!pip install --upgrade tensorflow
!pip install --upgrade librosa
import glob

print('Copying checkpoints from GCS...')
!rm -r /content/onsets-frames
!mkdir /content/onsets-frames
!gsutil -q -m cp -R gs://magentadata/models/onsets_frames_transcription/*checkpoint*.zip /content/onsets-frames/
!unzip -o /content/onsets-frames/maestro_checkpoint.zip -d /content/onsets-frames/maestro
MAESTRO_CHECKPOINT_DIR = '/content/onsets-frames/maestro/train'
!unzip -o /content/onsets-frames/e-gmd_checkpoint.zip -d /content/onsets-frames/e-gmd
EGMD_CHECKPOINT_DIR = '/content/onsets-frames/e-gmd'
  
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev ffmpeg  
!pip install pyfluidsynth pretty_midi

!pip install -qU magenta

#Feature Extraction
!pip install music21
!pip install MIDIFile

     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 1.9MB 13.1MB/s 
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.7/dist-packages (2.5.0)
Requirement already up-to-date: librosa in /usr/local/lib/python3.7/dist-packages (0.8.1)
Copying checkpoints from GCS...
rm: cannot remove '/content/onsets-frames': No such file or directory
Archive:  /content/onsets-frames/maestro_checkpoint.zip
   creating: /content/onsets-frames/maestro/train/
  inflating: /content/onsets-frames/maestro/train/model.ckpt.data-00000-of-00001  
  inflating: /content/onsets-frames/maestro/train/model.ckpt.meta  
  inflating: /content/onsets-frames/maestro/train/model.ckpt.index  
  inflating: /content/onsets-frames/maestro/train/checkpoint  
Archive:  /content/onsets-frames/e-gmd_checkpoint.zip
  inflating: /content/onsets-frames/e-gmd/checkpoint  
  inflating: /content/onsets-frames/e-gmd/model.ckpt-569400.data-00000-of-00001  
  inflating: /content/on

## Unmix Separation

In [ ]:
import os
for song in songs:
  os.chdir(root + song)
  isSuccess = os.system('umx "' + song + '.wav" ' + '--model umxhq')
  if(isSuccess != 0):
    print("Something went wrong separating files, restart runtime and try again")

FileNotFoundError: ignored

## Midi Creation through Magenta

In [ ]:
#@title Select Magenta Model
model_type = "MAESTRO (Piano)" #@param ["MAESTRO (Piano)", "E-GMD (Drums)"]


In [ ]:
#@title Initialize Magenta Model
import tensorflow.compat.v1 as tf
import librosa
import numpy as np

from google.colab import files

from magenta.common import tf_utils
from note_seq import audio_io
from magenta.models.onsets_frames_transcription import audio_label_data_utils
from magenta.models.onsets_frames_transcription import configs
from magenta.models.onsets_frames_transcription import constants
from magenta.models.onsets_frames_transcription import data
from magenta.models.onsets_frames_transcription import infer_util
from magenta.models.onsets_frames_transcription import train_util
import note_seq
from note_seq import midi_io
from note_seq import sequences_lib

tf.disable_v2_behavior()

## Define model and load checkpoint
## Only needs to be run once.

if model_type.startswith('MAESTRO'):
  config = configs.CONFIG_MAP['onsets_frames']
  hparams = config.hparams
  hparams.use_cudnn = False
  hparams.batch_size = 1
  checkpoint_dir = MAESTRO_CHECKPOINT_DIR
elif model_type.startswith('E-GMD'):
  config = configs.CONFIG_MAP['drums']
  hparams = config.hparams
  hparams.batch_size = 1
  checkpoint_dir = EGMD_CHECKPOINT_DIR
else:
  raise ValueError('Unknown Model Type')

examples = tf.placeholder(tf.string, [None])

dataset = data.provide_batch(
    examples=examples,
    preprocess_examples=True,
    params=hparams,
    is_training=False,
    shuffle_examples=False,
    skip_n_initial_records=0)

estimator = train_util.create_estimator(
    config.model_fn, checkpoint_dir, hparams)

iterator = tf.data.make_initializable_iterator(dataset)
next_record = iterator.get_next()

In [ ]:
#@title Magenta midi creation { form-width: "150px" }
from pydub import AudioSegment
from google.colab import files
#uploaded = files.upload()
def transcription_data(params):
  del params
  return tf.data.Dataset.from_tensors(sess.run(next_record))
#print(f'uploaded {uploaded}')
#for fn in uploaded.keys():
directory = []
# for d in directory:
#   spl = d + '/Spleeter/'
#   song = os.listdir(spl)
#   print(f'arr: {song}')
stems = ['vocals.wav', 'bass.wav', 'other.wav', 'drums.wav']
#spl = './Vanilla-Ice-Ice-Ice-Baby-_Official-Music-Video__umxhq/'
for song in songs:
  os.chdir(root+song)
  for stem in stems:
    to_process = []
    with open(song + '_umxhq/'+stem, mode='rb') as file: 
      wav_data = file.read()
    #print('User uploaded file "{name}" with length {length} bytes'.format( name=fn, length=len(uploaded[fn])))
    #wav_data = uploaded[fn]
    #print(f'wav data {wav_data}')
    example_list = list(audio_label_data_utils.process_record(
            wav_data=wav_data,
            sample_rate=hparams.sample_rate,
            ns=note_seq.NoteSequence(),
            example_id=stem,
            min_length=0,
            max_length=-1,
            allow_empty_notesequence=True))
    assert len(example_list) == 1
    to_process.append(example_list[0].SerializeToString())
    
    #print('Processing complete for', fn)
    
    sess = tf.Session()

    sess.run([
        tf.initializers.global_variables(),
        tf.initializers.local_variables()
    ])

    sess.run(iterator.initializer, {examples: to_process})

    input_fn = infer_util.labels_to_features_wrapper(transcription_data)

    prediction_list = list(
      estimator.predict(
          input_fn,
          yield_single_examples=False))
    assert len(prediction_list) == 1

    sequence_prediction = note_seq.NoteSequence.FromString(
        prediction_list[0]['sequence_predictions'][0])

    # Ignore warnings caused by pyfluidsynth
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning) 
    #print(f'sequence_prediction: {prediction_list[0]['sequence_predictions'][0][0]}')
    

    note_seq.plot_sequence(sequence_prediction)
    note_seq.play_sequence(sequence_prediction, note_seq.midi_synth.fluidsynth,
                    colab_ephemeral=False)
    !ls
    !mkdir {'./midi'}
    midi_filename = ('./midi/' + stem[:-3] + 'mid')
    print(f'midi_filename:  {midi_filename}')
    midi_io.sequence_proto_to_midi_file(sequence_prediction, midi_filename)





In [ ]:
#@title determine key and transpose to C/Am
import music21
import os
scale_normalisation_ignores = ['drums.mid']
for song in songs:
  os.chdir(root + song)
  cur_path = root + song + '/midi'
  file_names = os.listdir('./midi/')


  #remove the drums.mid
  for item in scale_normalisation_ignores:
    if item in file_names:
      file_names.remove(item)
  file_names2 = file_names.copy()

  #remove C_*.mid files
  for item in file_names2: 
    if item.startswith('C_'):
      file_names.remove(item)

  print(f'arr: {file_names}')
  highestKey = None
  highestProbability = 0
  #for stem in song:
  for midi in file_names:
    score = music21.converter.parse('./midi/' + midi)
    key = score.analyze('Krumhansl')
    print(f'Results for {song + midi}')
    print(f'Key {key} has probability {key.correlationCoefficient}')
    if (key.correlationCoefficient > highestProbability):
      highestKey = key
      highestProbability = key.correlationCoefficient
    for altKey in key.alternateInterpretations:
      if(altKey.correlationCoefficient < 0.5):
        break
      print(f'Key {altKey} has probability {altKey.correlationCoefficient}')

  print(f'Most likely key for song {song[2:-14]} is {highestKey} with probability {highestProbability}')

  print(f'Transposing midi to C/Am')
  majors = dict([("A-", 4),("A", 3),("B-", 2),("B", 1),("C", 0),("D-", -1),("D", -2),("E-", -3),("E", -4),("F", -5),("G-", 6),("G", 5)])
  minors = dict([("A-", 1),("A", 0),("B-", -1),("B", -2),("C", -3),("D-", -4),("D", -5),("E-", 6),("E", 5),("F", 4),("G-", 3),("G", 2)])
  key = highestKey
  #    print key.tonic.name, key.mode
  if key.mode == "major":
      halfSteps = majors[key.tonic.name]
      
  elif key.mode == "minor":
      halfSteps = minors[key.tonic.name]

  print(f'Transposing by {halfSteps}  semi-tones')
  for midi in file_names:
    score = music21.converter.parse('./midi/' + midi)
    newscore = score.transpose(halfSteps)
    #print key.tonic.name, key.mode
    newFileName = "./midi/C_" + midi
    newscore.write('midi',newFileName)
    print(f'Transposed midi file saved at {newFileName} ')
    



In [ ]:
#@title normalise midi to one octave
import os
for song in songs:
  os.chdir(root + song)
  file_names = os.listdir('./midi/')

  #remove the drums.mid
  for item in scale_normalisation_ignores:
    if item in file_names:
      file_names.remove(item)
  file_names2 = file_names.copy()

  #remove C_*.mid files
  for item in file_names2: 
    if not item.startswith('C_'):
      file_names.remove(item)

  for midi in file_names:
    print(f'Squish {midi} to one octave')
    sequence_prediction = note_seq.midi_file_to_note_sequence('./midi/' + midi)
    for i in range(len(sequence_prediction.notes)):
      sequence_prediction.notes[i].pitch = sequence_prediction.notes[i].pitch % 12
    
    note_seq.note_sequence_to_midi_file(sequence_prediction, './midi/' + '0' + midi)





## Feature Extraction

In [ ]:
#@title Helper Functions
import numpy as np
import scipy.fftpack as fft
def get_fft(rate, data):
    sample_length = len(data)
    k = np.arange(sample_length)
    period = sample_length / rate
    freqs = (k / period)[range(sample_length // 2)] #right-side frequency range
    fourier = abs(fft.fft(data * np.hanning(sample_length)) / sample_length) #normalized, not clipped
    fourier = fourier[range(sample_length // 2)] #clip to right-side
    power = np.power(fourier, 2.0)
    return power, freqs

def average_frequency(rate, data):
    power, freqs = get_fft(rate, data)
    #plt.plot(freqs, power)
    #plt.show()
    return sum(power * freqs) / sum(power)

def median_frequency(rate, data):
    power, freqs = get_fft(rate, data)
    i = 0
    highestValue = power[0]
    highestIndex = 0
    while i < len(power):
        if (power[i] > highestValue):
            highestIndex = i
            highestValue = power[i]
        i += 1
    return freqs[highestIndex]

In [ ]:
!pip install SpeechRecognition

In [ ]:
#@title Key, Tempo, Frequency components
import music21
import os
import librosa
file_names = ['bass.mid', 'vocals.mid', 'other.mid']

for song in songs:
  os.chdir(root + song)
  highestKey = None
  highestProbability = 0
  print(f'Features for song {song} are:')
  for midi in file_names:
    score = music21.converter.parse('./midi/' + midi)
    key = score.analyze('Krumhansl')
    #print(f'Results for {song_path + midi}')
    #print(f'Key {key} has probability {key.correlationCoefficient}')
    if (key.correlationCoefficient > highestProbability):
      highestKey = key
      highestProbability = key.correlationCoefficient
    for altKey in key.alternateInterpretations:
      if(altKey.correlationCoefficient < 0.5):
        break

  #print(f'Most likely key is {highestKey} with probability {highestProbability}')
  y, sample_rate = librosa.load(song + '.wav')

  # 3. Run the default beat tracker
  tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sample_rate)


  sample_rate_string = f'Sample rate is {sample_rate}\n'
  key_string = f'Key is {highestKey}\n'
  tempo_string = 'Tempo is {:.2f} beats per minute\n'.format(tempo)
  av_freq_string = 'Average frequency is {:.2f}Hz\n'.format(average_frequency(sr, y))
  median_freq_string = 'Median frequency is {:.2f}Hz\n'.format(median_frequency(sr, y))
  print
  print(sample_rate_string)
  print(key_string)
  print(tempo_string)
  print(av_freq_string)
  print(median_freq_string)
  #Save file
  f = open("./Features.txt", "w")
  f.write("Transcription: " + transcript)
  f.write(sample_rate_string)
  f.write(key_string)
  f.write(tempo_string)
  f.write(av_freq_string)
  f.write(median_freq_string)
  f.close()




In [ ]:
%cd /content/gdrive/MyDrive/audio_files/Unmix Demo/Vanilla-Ice-Ice-Ice-Baby/
!ls

/content/gdrive/.shortcut-targets-by-id/16Ktb_bUzQ3didLw2EAmzipUs8iqQYF2Q/audio_files/Unmix Demo/Vanilla-Ice-Ice-Ice-Baby
Features.txt  Vanilla-Ice-Ice-Ice-Baby_umxhq
midi	      Vanilla-Ice-Ice-Ice-Baby.wav


In [ ]:
from MIDI import MIDIFile
c=MIDIFile(root+songs[0] + '/midi/0C_vocals.mid')
c.parse()
print(str(c))

Format 1 nTracks 2 division 1024
	Track 0 of length 0
	Track 1 of length 0
